In [1]:
sc =spark.sparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import BinaryClassificationMetrics
tf= HashingTF(numFeatures= 10000)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1585916550929_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
labelled = sc.pickleFile("s3://tutorial-emr-2020/pickleLabelled")
emails = labelled.map(lambda x : x[1])
emails = emails.map(lambda x: (x[0],x[1].replace("\n"," ")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#divido
spam = emails.filter(lambda x : x[0] ==1)
ham = emails.filter(lambda x : x[0] ==0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#trasformo le parole in features
spamFeatures=spam.map(lambda x:(x[0],tf.transform(x[1].split(" "))))
hamFeatures=ham.map(lambda x:(x[0],tf.transform(x[1].split(" "))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#divido in train e test
trainSpam,testSpam = spamFeatures.randomSplit([0.8,0.2])
trainHam,testHam = hamFeatures.randomSplit([0.8,0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#tengo le mail
trainSpam = trainSpam.map(lambda x : x[1])
trainHam = trainHam.map(lambda x : x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
spamExamples = trainSpam.map(lambda features: LabeledPoint(1, features))
hamExamples = trainHam.map(lambda features: LabeledPoint(0, features))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
testData = testSpam.union(testHam)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
trainingData = spamExamples.union(hamExamples)
trainingData.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

UnionRDD[7] at union at NativeMethodAccessorImpl.java:0

In [10]:
model = LogisticRegressionWithLBFGS.train(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
predictionAndLabels = testData.map(lambda x: (float(model.predict(x[1])), x[0]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1.0, 1)

In [12]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
precision = metrics.precision()
print("Precision = %s" % precision)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'BinaryClassificationMetrics' object has no attribute 'precision'
Traceback (most recent call last):
AttributeError: 'BinaryClassificationMetrics' object has no attribute 'precision'



In [ ]:
sc.stop()